# Connecting to BigQuery
## resource with problem statement of this demo
- https://youtu.be/ieaqfU1BwJ8
## create queries to filter data, extract features, and clean where possible

### below is an **example** query
- here a query is made and stored in dataframe
- more tags, and api documentation are provided here: https://googleapis.dev/python/bigquery/latest/magics.html



In [ ]:
%load_ext google.cloud.bigquery

### example query to strip **html** , and replace "|" with "," for delimiting

### save query to python pandas dataframe
- more data manipulation and rangling can be done in python if needed

In [ ]:
type(dataframe)

In [ ]:
dataframe

### python string formating -- **data preperation for model training**
- code below removes all "," and new lines in article
- then the tags are filtered and delimited by ","
- this is done because training **csv** file is delimited by ","

In [ ]:
#creating deep copy to test with
test_dataframe = dataframe.copy(deep=True)

# items selected for training
interest_items = ["python", "reactjs", "pandas", "numpy", "javascript","sql", "jquery", "django", "php"]
with open("file_write.csv", "w") as file_write:
    for index, row in test_dataframe.iterrows():

        #csv format requires delimiting by comma, appending comma at end of question
        test_dataframe.at[index, 'article'] = test_dataframe.at[index, 'article'].replace(',','')
        test_dataframe.at[index, 'article'] = test_dataframe.at[index, 'article'].replace('\n','')
        test_dataframe.at[index, 'article'] = test_dataframe.at[index, 'article'] + ','

        clean_list = ""
        #create iterable list of tags
        row['tags'] = row['tags'].split(',') 
        for tag in row['tags']:
            if tag in interest_items:
                clean_list += tag + "," 
        # # delete rows which don't contain interest items
        if not clean_list:
            test_dataframe.drop(index, inplace=True)

        else:
            clean_list = clean_list[:-1]
            test_dataframe.at[index, 'tags'] = clean_list
            # writing datafram with filtered valuews into comma delimited csv
            file_write.write(test_dataframe.at[index, 'article'] + clean_list + "\n")
        

### note the difference between original and filtered dataframe
- test_dataframe == filtered
- dataframe == original
- test_dataframe even has dropped certain rows, due to a lack of matched interest items

In [ ]:
test_dataframe.head()

In [ ]:
dataframe.head()

# After filtering and writing dataframe to CSV, upload to **Google CLoud Storage** (GCS)
## the second cell runs a shell command. 
- use the "!" charector to initiate shell command from **jupitor notebook**

In [ ]:
!gsutil cp file_write.csv gs://[BUCKET-NAME]/

# Creating a model, training and deploying
## created based off of this resourece:https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/automl-text-classification.ipynb

In [ ]:
#pip install required libraries
! pip install {USER_FLAG} --upgrade google-cloud-aiplatform google-cloud-storage jsonlines

In [ ]:
from google.cloud import aiplatform, storage, bigquery
from google.protobuf import json_format

REGION = "[REGION]"
PROJECT_ID = "[PROJECT_ID]"
BUCKET_NAME = "[BUCKET_NAME]"
FILE_NAME = "[FILE_NAME]"

src_uris ="gs://{}/{}".format(BUCKET_NAME, FILE_NAME)
display_name = "[DISPLAY_NAME -- your choice of naming"

aiplatform.init(project=PROJECT_ID, location=REGION)

## create text multi label dataset

In [ ]:
#creating text dataset
ds = aiplatform.TextDataset.create(
    display_name=display_name,
    gcs_source=src_uris,
    import_schema_uri=aiplatform.schema.dataset.ioformat.text.multi_label_classification,
    sync=True,
)

## pull dataset id of most recent dataset created

In [ ]:
dataset_id = ds.resource_name.split("/")[-1]
text_dataset = aiplatform.TextDataset(dataset_id)

## create training job description

In [ ]:
# Define the training job
training_job_display_name = "stack_question_training" # change to your liking
job = aiplatform.AutoMLTextTrainingJob(
    display_name=training_job_display_name,
    prediction_type="classification",
    multi_label=True,
)

## run model training
- this took about 4 hours in this test

In [ ]:
model_display_name = "stack_question_model" # change to your liking

# Run the training job
model = job.run(
    dataset=text_dataset,
    model_display_name=model_display_name,
    training_fraction_split=0.7,
    validation_fraction_split=0.2,
    test_fraction_split=0.1,
    sync=True,
)

##  deploy model

In [ ]:
deployed_model_display_name = "model-endpoint-stack-questions" # change to your liking

endpoint = model.deploy(
    deployed_model_display_name=deployed_model_display_name, sync=True
)

# once model is deployed: 
- find model endpoint id through vertex ai "endpoints" page on **Google Cloud Console**
- example: "[GOOGLE_CLOUD_CONSOLE]/vertex-ai/endpoints?project=[YOUR PROJECT HERE]"
- funciton below is pulled from: https://github.com/googleapis/python-aiplatform/blob/main/samples/snippets/prediction_service/predict_text_classification_single_label_sample.py
- **this function** will test your deployed model and is an example of how to use your model to recieve predictions

## visualize performance of your model
- navigate to "model" page of your vertex ai dashboard
- here you can inspect the performance of each of your models


In [ ]:
def predict_text_classification_multi_lable(
    project: str,
    endpoint_id: str,
    content: str,
    location: str ,
    api_endpoint: str = "[your model location]-aiplatform.googleapis.com",
):
    api_endpoint = location + "-aiplatform.googleapis.com"
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    instance = predict.instance.TextClassificationPredictionInstance(
        content=content,
    ).to_value()
    instances = [instance]
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    # See gs://google-cloud-aiplatform/schema/predict/prediction/text_classification.yaml for the format of the predictions.
    predictions = response.predictions
    for prediction in predictions:
        dictionary = dict(prediction)
        print(" prediction:")
        for index in range(len(dictionary['displayNames'])):
            print("\t", dictionary['displayNames'][index], ": " , dictionary['confidences'][index])
    return dictionary 
            

In [ ]:
results =  predict_text_classification_multi_lable(
            "[PROJECT_ID]",
            endpoint_id="[YOUR ENDPOINT ID]",
            location="[LOCATION FOR YOUR ENDPOINT]",
            content= "CONTENT YOU WANT TO TEST"
        ) 